# Workshop 3
Fabián González Vera A01367585
17/08/2023

## 2 The Central Limit Theorem

2.2 Monte Carlo simulation

In [4]:
from symbulate import *
import matplotlib
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
import pandas as pd
import numpy as np

In [14]:
# Getting price data and selecting adjusted price columns:
sprices = yf.download(tickers=['AMD','INTC'],start="2017-01-01",interval="1mo")
sprices=sprices['Adj Close']
sprices

[*********************100%%**********************]  2 of 2 completed


,AMD,INTC
Date,,
2017-01-01,10.370000,30.562130
2017-02-01,14.460000,30.047506
2017-03-01,14.550000,30.153332
2017-04-01,13.300000,30.220203
2017-05-01,11.190000,30.186773
...,...,...
2023-04-01,89.370003,30.822439
2023-05-01,118.209999,31.199532
2023-06-01,113.910004,33.320126


In [18]:
sprices["AMDr"] = (sprices["AMD"] / sprices["AMD"].shift(1)) - 1
sprices["INTCr"] = (sprices["INTC"] / sprices["INTC"].shift(1)) - 1
sprices

,AMD,INTC,AMDr,INTCr
Date,,,,
2017-01-01,10.370000,30.562130,NaN,NaN
2017-02-01,14.460000,30.047506,0.394407,-0.016839
2017-03-01,14.550000,30.153332,0.006224,0.003522
2017-04-01,13.300000,30.220203,-0.085911,0.002218
2017-05-01,11.190000,30.186773,-0.158647,-0.001106
...,...,...,...,...
2023-04-01,89.370003,30.822439,-0.088154,-0.049281
2023-05-01,118.209999,31.199532,0.322703,0.012234
2023-06-01,113.910004,33.320126,-0.036376,0.067969


In [19]:
sprices = sprices.dropna()
sprices

,AMD,INTC,AMDr,INTCr
Date,,,,
2017-02-01,14.460000,30.047506,0.394407,-0.016839
2017-03-01,14.550000,30.153332,0.006224,0.003522
2017-04-01,13.300000,30.220203,-0.085911,0.002218
2017-05-01,11.190000,30.186773,-0.158647,-0.001106
2017-06-01,12.480000,28.415361,0.115282,-0.058682
...,...,...,...,...
2023-04-01,89.370003,30.822439,-0.088154,-0.049281
2023-05-01,118.209999,31.199532,0.322703,0.012234
2023-06-01,113.910004,33.320126,-0.036376,0.067969
